In [1]:
import jax.numpy as jnp
import hj_reachability as hj
import jax
from torch2jax import j2t, t2j

In [2]:
from dynamics.dynamics_hjr import Quad2DAttitude as Quad2DAttitude_hjr

In [3]:
from dynamics.dynamics import Quad2DAttitude

In [4]:
dynamics = Quad2DAttitude(9.81, 0.75, 5.0, 15.0)

/home/sander/dev/deepreach/utils/boundary_functions.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.min_val = torch.tensor(min_val).to(device)[torch.newaxis]
/home/sander/dev/deepreach/utils/boundary_functions.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.max_val = torch.tensor(max_val).to(device)[torch.newaxis]
/home/sander/dev/deepreach/utils/boundary_functions.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.center = torch.tensor(center).to(device)[torch.newaxis]
/home/sander/dev/deepreach/utils/boundary_

In [5]:
dynamics_hjr = Quad2DAttitude_hjr(dynamics, 9.81, 0.75, 5.0, 15.0)

In [6]:
state_test_range = t2j(dynamics_hjr.torch_dynamics.state_bounds)

In [7]:
state_domain = hj.sets.Box(lo=state_test_range[:, 0], hi=state_test_range[:, 1])
grid_resolution = tuple([51]) * state_test_range.shape[0]

In [8]:
grid = hj.Grid.from_lattice_parameters_and_boundary_conditions(state_domain, grid_resolution)

In [9]:
from utils import boundary_functions
space_boundary = boundary_functions.BoundaryJAX([0, 1, 2, 3], jnp.array([-4.5, 0.0, -3.0, -3.0]), 
                                            jnp.array([4.5, 2.5, 3.0, 3.0]))
circle = boundary_functions.CircleJAX([0, 1], 0.5, jnp.array([2.0, 1.5]))
rectangle = boundary_functions.RectangleJAX([0, 1], jnp.array([-2.0, 0.5]), jnp.array([-1.0, 1.5]))
sdf_function = boundary_functions.build_sdf_jax(space_boundary, [circle, rectangle])
sdf_values = hj.utils.multivmap(sdf_function, jnp.arange(grid.ndim))(grid.states).squeeze()
backwards_reachable_tube = lambda obstacle: (lambda t, x: jnp.minimum(x, obstacle))
solver_settings = hj.SolverSettings.with_accuracy("very_high", 
                                                  value_postprocessor=backwards_reachable_tube(sdf_values))

In [10]:
min_time = 0.0
max_time = -1.0
times = jnp.linspace(min_time, max_time, 5)

In [11]:
values = hj.solve(solver_settings, dynamics_hjr, grid, times, sdf_values, progress_bar=False)

In [12]:
values.shape

(5, 51, 51, 51, 51)

In [13]:
import matplotlib.pyplot as plt

In [14]:
plt.contourf(grid.coordinate_vectors[0], grid.coordinate_vectors[1], values[0][:,:,25,25])